In [19]:
import yfinance as yf
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return, ema_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [20]:
# Define the tickers and date range
# tickers = ["CGL.TO", "XUS.TO", "XGB.TO", "RY.TO", "WMT.NE", "JNJ.NE", "APLY.NE", "VISA.NE", "TSLA.NE", "BRK.NE"]  
tickers = ["CGL.TO", "XUS.TO", "XGB.TO", "RY.TO", "WMT", "JNJ", "AAPL", "V", "TSLA", "BRK-B"]  
start_date = "2019-01-01"
end_date = "2024-08-04"  # Today's date

# Fetch data from Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  10 of 10 completed


In [21]:
# Extract the 'Close' prices
prices = data['Close']

# Ensure the index is in the correct format
prices.index = pd.to_datetime(prices.index)

# Drop any missing values
prices = prices.dropna(axis=0)

# Display the first few rows of the data
prices

Ticker,AAPL,BRK-B,CGL.TO,JNJ,RY.TO,TSLA,V,WMT,XGB.TO,XUS.TO
Date,,,,,,,,,,
2019-01-02,39.480000,202.800003,10.93,127.750000,93.620003,20.674667,132.919998,31.113333,21.309999,42.464931
2019-01-03,35.547501,191.660004,11.03,125.720001,92.570000,20.024000,128.130005,30.953333,21.459999,41.082165
2019-01-04,37.064999,195.199997,10.99,127.830002,94.139999,21.179333,133.649994,31.146667,21.350000,42.244488
2019-01-07,36.982498,196.910004,11.00,127.010002,93.910004,22.330667,136.059998,31.513332,21.330000,42.224449
2019-01-08,37.687500,196.309998,10.96,129.960007,93.739998,22.356667,136.800003,31.733334,21.309999,42.585171
...,...,...,...,...,...,...,...,...,...,...
2024-07-29,218.240005,438.309998,18.99,158.559998,152.690002,232.100006,261.600006,69.620003,19.219999,93.839996
2024-07-30,218.800003,441.260010,19.16,161.330002,153.899994,222.619995,263.100006,69.190002,19.280001,93.349998
2024-07-31,222.080002,438.500000,19.51,157.850006,154.279999,232.070007,265.670013,68.639999,19.340000,94.550003


In [22]:
# Calculate expected returns and sample covariance matrix
# mu = mean_historical_return(prices)
mu = ema_historical_return(prices, frequency=252)
S = CovarianceShrinkage(prices).ledoit_wolf()

In [23]:
# Optimize the portfolio
ef = EfficientFrontier(mu, S)

# Add L2 regularization to the optimization problem
ef.add_objective(objective_functions.L2_reg, gamma=0.1)

weights = ef.max_sharpe()
# weights = ef.min_volatility()

# Get the discrete allocation of each asset
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Print the expected annual return, annual volatility and Sharpe ratio
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.17427), ('BRK-B', 0.15143), ('CGL.TO', 0.24949), ('JNJ', 0.0), ('RY.TO', 0.06128), ('TSLA', 0.0), ('V', 0.0), ('WMT', 0.24718), ('XGB.TO', 0.0), ('XUS.TO', 0.11636)])
Expected annual return: 24.3%
Annual volatility: 14.4%
Sharpe Ratio: 1.54


/home/maxzhang/miniconda3/envs/MIE377/lib/python3.11/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


(0.2430639679145541, 0.1444285518528693, 1.5444589387131116)

In [26]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = da.lp_portfolio()
print(allocation)

{'AAPL': 79, 'BRK-B': 35, 'CGL.TO': 1286, 'RY.TO': 41, 'WMT': 361, 'XGB.TO': 5, 'XUS.TO': 127}
